In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import random as rand
%matplotlib inline
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, hp

In [4]:
trainset = dsets.CIFAR10('./data', train=True, download=True, transform=transforms.ToTensor())
testset = dsets.CIFAR10('./data', train=False, download=True, transform=transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [5]:
if torch.cuda.is_available():
    avDev = torch.device("cuda")
else:
    avDev = torch.device("cpu")

print(avDev)

cuda


In [6]:
trainset[1][0]

tensor([[[0.6039, 0.4941, 0.4118,  ..., 0.3569, 0.3412, 0.3098],
         [0.5490, 0.5686, 0.4902,  ..., 0.3765, 0.3020, 0.2784],
         [0.5490, 0.5451, 0.4510,  ..., 0.3098, 0.2667, 0.2627],
         ...,
         [0.6863, 0.6118, 0.6039,  ..., 0.1647, 0.2392, 0.3647],
         [0.6471, 0.6118, 0.6235,  ..., 0.4039, 0.4824, 0.5137],
         [0.6392, 0.6196, 0.6392,  ..., 0.5608, 0.5608, 0.5608]],

        [[0.6941, 0.5373, 0.4078,  ..., 0.3725, 0.3529, 0.3176],
         [0.6275, 0.6000, 0.4902,  ..., 0.3882, 0.3137, 0.2863],
         [0.6078, 0.5725, 0.4510,  ..., 0.3216, 0.2745, 0.2706],
         ...,
         [0.6549, 0.6039, 0.6275,  ..., 0.1333, 0.2078, 0.3255],
         [0.6039, 0.5961, 0.6314,  ..., 0.3647, 0.4471, 0.4745],
         [0.5804, 0.5804, 0.6118,  ..., 0.5216, 0.5255, 0.5216]],

        [[0.7333, 0.5333, 0.3725,  ..., 0.2784, 0.2784, 0.2745],
         [0.6627, 0.6039, 0.4627,  ..., 0.3059, 0.2431, 0.2392],
         [0.6431, 0.5843, 0.4392,  ..., 0.2510, 0.2157, 0.

In [7]:
len(trainset)

50000

In [8]:
# STEP 2: MAKING DATASET ITERABLE
 
batch_size = 100
n_iters = 8000
num_epochs = n_iters / (len(trainset) / batch_size)
num_epochs = int(num_epochs)
 
train_loader = torch.utils.data.DataLoader(dataset=trainset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=testset, 
                                          batch_size=batch_size, 
                                          shuffle=False)
 

In [9]:
'''
STEP 3: CREATE MODEL CLASS
'''
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear1 = nn.Linear(input_dim, 3000)
        self.linear2 = nn.Linear(3000, output_dim)
        self.relu = nn.ReLU()
     
    def forward(self, x):
        out = self.relu(self.linear1(x))
        out = self.linear2(out)
        return out

In [10]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 3*32*32
output_dim = 10
 
model = LogisticRegressionModel(input_dim, output_dim)

In [11]:
#  USE GPU FOR MODEL  #
#######################
 
model.to(avDev)
 
'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss().to(avDev)


In [12]:
'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.05
 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [38]:
def hyperopt_optimizer(outputs, labels):  
    def f(outputs, labels=labels):
#         return criterion(outputs, labels)
        return (outputs - labels).pow(2).sum()

    space = hp.uniform('learning_rate', -1, 1)

    best = fmin(
        fn=f,  # "Loss" function to minimize
        space=space,  # Hyperparameter space
        algo=tpe.suggest,  # Tree-structured Parzen Estimator (TPE)
        max_evals=1000  # Perform 1000 trials
    )
    return best

In [39]:
'''
STEP 7: TRAIN THE MODEL
'''

loss_save = np.empty(n_iters)

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
         
        images = images.view(-1, 3*32*32).to(avDev)
        labels = labels.to(avDev)
         
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        outputs = model(images)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)#
         
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
#            optimizer.step()
        hyperopt_optimizer(outputs, labels)
           
        #Save Loss    
        
        loss_save[iter-1] = loss.item()
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                images = images.view(-1, 3*32*32).to(avDev)
                 
                # Forward pass only to get logits/output
                outputs= model(images)
                 
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                 
                # Total number of labels
                total += labels.size(0)
                 
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions
                correct += (predicted.cpu() == labels.cpu()).sum().float()
             
            accuracy = 100. * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

  0%|          | 0/1000 [00:00<?, ?it/s, best loss: ?]


TypeError: iteration over a 0-d tensor

In [ ]:
correct = 0
total = 0
model.to(avDev)

with torch.no_grad():
    for data in train_loader:
        images, labels = data
        images = images.view(-1, 3*32*32).to(avDev)
        labels = labels.to(avDev)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the  train images: %d %%' % (
    100 * correct / total))

In [ ]:
print(images.size())

In [ ]:
outputs.size()

In [ ]:
labels.size()

In [ ]:
ax = sns.lineplot(data = loss_save)
ax.set(xlabel = "Iterations", ylabel="Loss")
plt.show()

In [ ]:
model.to(avDev)

with torch.no_grad():
    for data in train_loader:
        images, labels = data
        images = images.view(-1, 3*32*32).to(avDev)
        labels = labels.to(avDev)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
confusion_matrix(predicted.cpu(), labels.cpu())

In [ ]:

images, labels = data
# images = images.view(-1, 3*32*32)
# labels = labels
model = LogisticRegressionModel(input_dim, output_dim).to(avDev)

def hyperopt_train_test(images):
# def hyperopt_train_test(input_dim, output_dim):
#     clf = LogisticRegressionModel(input_dim, output_dim)
#     outputs = model(images)
#     clf = criterion(outputs, labels)
    clf = model(images)
    return cross_val_score(clf, images, labels).mean()

space = {
    'learning_rate': hp.choice('learning_rate', range(-1,1))
}

def f(params):
    acc = hyperopt_train_test(images)
    return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, space, algo=tpe.suggest, max_evals=100, trials=trials)
print ('best:')
print(best)